In [143]:
import sqlite3 
import xlrd
import geocoder
import os
import re
import datetime

from xlrd import xldate_as_tuple

# 1. Build Database

In [144]:
con_db = sqlite3.connect('Data_Challenge.db')
cur = con_db.cursor()

# 2. Customer.xlsx handled individually since geocoding issue
   

#### throught the geocoder library we can easily access the service from so many geocoding API provider
#### at this time I'm using the ArcGIS

In [145]:
def single_insert(file_name):
    con_db = sqlite3.connect('Data_Challenge.db')
    cur = con_db.cursor()
    
    data = xlrd.open_workbook(file_name)
    table = data.sheets()[0]
    
    cur.execute('CREATE TABLE IF NOT EXISTS Customers(Customer_Name TEXT, Country TEXT, Region TEXT, Longtitude REAL, Latitude REAL, Customer_Number INTEGER)')
    
    for i in range(1, table.nrows):
        Customer_Name = str(table.row_values(i)[0])
        Country = str(table.row_values(i)[1])
        Region = str(table.row_values(i)[2])
        lnglat = geocoder.arcgis(Country + ',' + Region)
        Longtitude = lnglat.lng
        Latitude = lnglat.lat
        Customer_Number = str(table.row_values(i)[3])
        data_entry(Customer_Name, Country, Region, Longtitude, Latitude, Customer_Number)    

In [146]:
def data_entry(Customer_Name, Country, Region, Longtitude, Latitude, Customer_Number):
    con_db = sqlite3.connect('Data_Challenge.db')
    cur = con_db.cursor()
    cur.execute("INSERT INTO Customers(Customer_Name, Country, Region, Longtitude, Latitude, Customer_Number) VALUES (?, ?, ?, ?, ?, ?)",
          (Customer_Name, Country, Region, Longtitude, Latitude, Customer_Number))
    con_db.commit()
    

# 3. Batch Insertion for other sheets

### 1) insertion framework

In [147]:
def batch_insert(file_name):
    con_db = sqlite3.connect('Data_Challenge.db')
    cur = con_db.cursor()

    work_book = xlrd.open_workbook(file_name)
    sheet = work_book.sheet_by_index(0)
    start = sheet.row_values(0)[0]
    
    queries = build_2_query(file_name,sheet,start)
    
    # create table information
    cur.execute(queries[0])
    # insert sheet into the table
    sheet_insertion(sheet,queries[1])


### 2) build two queries: table_query for table creation, insert_query for sheet insertion.

In [148]:
def build_2_query(file_name,sheet,start):
    
    work_book = xlrd.open_workbook(file_name)

    table_query = 'CREATE TABLE IF NOT EXISTS ' + file_name[:-5] + '('
    insert_query_1 = 'INSERT INTO ' + file_name[:-5] + '('
    insert_query_2 = ' VALUES (?' 

    if start in Int_Cat:
        table_query += start + ' INTEGER'
        insert_query_1 += start
    if start in Real_Cat:
        table_query += start + ' REAL'
        insert_query_1 += start
    if start in Text_Cat:
        table_query += start + ' TEXT'
        insert_query_1 += start

    for row in sheet.row_values(0)[1:]:
        if row in Int_Cat:
            table_query += ', ' + row + ' INTEGER'
            insert_query_1 += ', ' + row 
            insert_query_2 += ', ?'
        if row in Real_Cat:
            table_query += ', ' + row + ' REAL'
            insert_query_1 += ', ' + row 
            insert_query_2 += ', ?'
        if row in Text_Cat:
            table_query += ', ' + row + ' TEXT'
            insert_query_1 += ', ' + row 
            insert_query_2 += ', ?'
    table_query += ')'
    insert_query = insert_query_1 + ')' + insert_query_2 + ')'
    return table_query, insert_query

### 3) sheet insertion function

In [149]:
def sheet_insertion(sheet,insert_query):
    con_db = sqlite3.connect('Data_Challenge.db')
    cur = con_db.cursor()
    for i in range(1, sheet.nrows):
        tup = []
        for j in range(0, sheet.ncols):
            if sheet.cell(i,j).ctype == 3:
                tup.append(date_convert(sheet.cell(i,j).value))
            else:
                tup.append(sheet.cell(i,j).value)
        tup = tuple(tup)
        cur.execute(insert_query,tup)
        con_db.commit()

### 4) Date Conversion: from timestamp to date

In [150]:
def date_convert(int_date):
    date = xldate_as_tuple(int_date,0)
    value = datetime.datetime.date(datetime.datetime(*date))
    return value

# 4. Files filter for execution

In [151]:
path = os.path.abspath('.') # absolute path

# keep those files who ends up with'.xlsx'
all_files = [f for f in os.listdir(path) if re.search(r'.xlsx$',f)]
# get rid those of temporary files(eg.'~$Customer.xlsx')
rest_files = [f for f in all_files if re.search(r'^[A-Z]',f) and f != 'Customers.xlsx']

# filter for different datatype

Int_Cat = ['Year','Month','Account','Product_Code',
           'Account','Invoice_Number','Year','Month','Quantity',
           'Product_Category_Code']
Real_Cat = ['Budget','Invoice_Amount','Standard_Price','Variable_Cost','Gross_Margin']
Text_Cat = ['Invoice_Date','Product','Product_Category']

# 5. Execution 

In [156]:
# 'Customers.xlsx' execution
single_insert('Customers.xlsx')

# 'Products.xlsx', 'Budgets.xlsx', 'Sales.xlsx' execution
for file in rest_files:
    batch_insert(file)

# database disconnection   
cur.close()
con_db.close()